In [1]:
import pandas  as pd

In [19]:
x = pd.read_csv("x_transposed_notNormalized.csv")
dfxTest= pd.read_csv("dfTest_transposed_notNormalized.csv")
y1= pd.read_csv("Y1_transposed.csv")
y2= pd.read_csv("Y2_transposed.csv")
y3= pd.read_csv("Y3_transposed.csv")
y4= pd.read_csv("Y4_transposed.csv")
y5= pd.read_csv("Y5_transposed.csv")


In [20]:
y = pd.DataFrame()
y["Washing Machine"] = y1.drop('Unnamed: 0',axis = 1)
y["Dishwasher"] = y2.drop('Unnamed: 0',axis = 1)
y["Tumble Dryer"] = y3.drop('Unnamed: 0',axis = 1)
y["Microwave"] = y4.drop('Unnamed: 0',axis = 1)
y["Kettle"] = y5.drop('Unnamed: 0',axis = 1)


In [21]:
x = x.drop('Unnamed: 0',axis = 1)
x['consumption_previous_day'] = x['0'].shift(1)
x.loc[0, 'consumption_previous_day'] = x.consumption_previous_day[x.consumption_previous_day.notnull()].values[0]

x["rolling_mean"] = x['0'].rolling(3).mean()
x["rolling_med"] = x['0'].rolling(3).median()
x["rolling_std"] = x['0'].rolling(3).std()



In [22]:
x

,0,consumption_previous_day,rolling_mean,rolling_med,rolling_std
0,180.0,180.0,NaN,NaN,NaN
1,180.0,180.0,NaN,NaN,NaN
2,180.0,180.0,180.000000,180.0,0.000000
3,181.0,180.0,180.333333,180.0,0.577350
4,180.0,181.0,180.333333,180.0,0.577350
...,...,...,...,...,...
22509355,366.5,378.0,373.166667,375.0,5.965177
22509356,376.0,366.5,373.500000,376.0,6.144103
22509357,367.0,376.0,369.833333,367.0,5.346338
22509358,353.0,367.0,365.333333,367.0,11.590226


In [23]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)
x['rolling_mean'] = imputer.fit_transform(x[['rolling_mean']])
x['rolling_med'] = imputer.fit_transform(x[['rolling_med']])
x['rolling_std'] = imputer.fit_transform(x[['rolling_std']])



In [24]:
x

,0,consumption_previous_day,rolling_mean,rolling_med,rolling_std
0,180.0,180.0,493.976997,493.148643,30.293880
1,180.0,180.0,493.976997,493.148643,30.293880
2,180.0,180.0,180.000000,180.000000,0.000000
3,181.0,180.0,180.333333,180.000000,0.577350
4,180.0,181.0,180.333333,180.000000,0.577350
...,...,...,...,...,...
22509355,366.5,378.0,373.166667,375.000000,5.965177
22509356,376.0,366.5,373.500000,376.000000,6.144103
22509357,367.0,376.0,369.833333,367.000000,5.346338
22509358,353.0,367.0,365.333333,367.000000,11.590226


In [25]:
import xgboost as xgb
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.4,random_state=42)

dtrain = xgb.DMatrix(X_train,label=y_train)
dtest = xgb.DMatrix(X_test,label=y_test)

model = xgb.XGBModel()
params = {
    'max_depth': 10,
    'eta': 0.1,
    'objective': 'binary:logistic',
    'eval_metric': 'error',
    'booster':'dart',
    'sample_type':'weighted',
    'normalize_type':'forest',
    'learning_rate':0.4,
    'tree_method':'hist',
    'n_jobs':500,
    'rate_drop':0.3,
    'skip_drop':0.2,

}

# Train model
evalist = [(dtrain, 'train'), (dtest, 'eval')]
model = xgb.train(params, dtrain,num_boost_round=30,verbose_eval=29,evals=evalist)

preds = model.predict(dtest) #Predictions
y_test = y_test.astype(float)
optimal_num_boost_round = model.best_iteration

print("Precision = {}".format(average_precision_score(y_test.values, preds, average='macro')))
print("Precision = {}".format(average_precision_score(y_test.values, preds, average='weighted')))


#Precision = 0.25674527434942446 YANIS
#Precision = 0.4107742871167801 YANIS


[0]	train-error:0.01100	eval-error:0.01098


In [8]:
print(optimal_num_boost_round)

29


In [9]:
dfxTest = dfxTest.drop('Unnamed: 0',axis = 1)
dfxTest['consumption_previous_day'] = dfxTest['0'].shift(1)
dfxTest.loc[0, 'consumption_previous_day'] = dfxTest.consumption_previous_day[dfxTest.consumption_previous_day.notnull()].values[0]

dfxTest["rolling_mean"] = dfxTest['0'].rolling(3).mean()
imputer = KNNImputer(n_neighbors=2)
dfxTest['rolling_mean'] = imputer.fit_transform(dfxTest[['rolling_mean']])


In [10]:
dFin = xgb.DMatrix(dfxTest)

preds_f = model.predict(dFin)

dfF = pd.DataFrame(preds_f, columns=["Washing Machine","Dishwasher","Tumble Dryer","Microwave","Kettle"])
dfF = dfF.rename_axis('Index')
dfF.to_csv("no_features_baseline.csv")

In [11]:
dfF = dfF.round(4)
dfF.to_csv("testing.csv")


In [12]:
dfF

,Washing Machine,Dishwasher,Tumble Dryer,Microwave,Kettle
Index,,,,,
0,0.4973,0.0012,0.0020,0.0001,0.0001
1,0.4973,0.0012,0.0020,0.0001,0.0001
2,0.0131,0.0177,0.0017,0.0001,0.0001
3,0.0236,0.0146,0.0018,0.0001,0.0001
4,0.0132,0.0142,0.0018,0.0001,0.0001
...,...,...,...,...,...
5374075,0.0331,0.0097,0.0215,0.0353,0.0200
5374076,0.0331,0.0097,0.0215,0.0353,0.0200
5374077,0.0331,0.0097,0.0215,0.0353,0.0200
